In [317]:
### Getting the total number of confirmed cases in the top 20 affected countries and all the states in the US.
### purpose: to run the cron-job every hour using HitchHiqe's resources to perform ridge based lambda regression 
### for further classification and improving the accuracy of the algorithm in better prediciton for the app.


from selenium import webdriver
import pandas as pd
import os
import re
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
import random
from datetime import datetime
import collections

In [318]:
URL = "https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6"

In [383]:
chrome_options = webdriver.ChromeOptions()
chrome_options.headless = True
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
br = webdriver.Chrome(executable_path=r"C:\Users/qasim/Downloads/chromedriver_win32_v80/chromedriver.exe",options=chrome_options)
br.get(URL)
time.sleep(3)

In [384]:
root = br.find_element_by_id("ember6")

In [321]:
root.screenshot("sample_image.png")

True

In [385]:
data = root.text.split("\n")[2:]

In [386]:
end = data.index("Last Updated at (M/D/YYYY)")

In [387]:
total_cases = data[1]

In [388]:
start = 1 + data.index("Confirmed Cases by Country/Region/Sovereignty")

In [389]:
data = data[start:end]

In [390]:
total_cases = int(total_cases.replace(",",""))

In [391]:
TIME = str(datetime.now())

In [392]:
total_cases

198178

In [393]:
case = {}

In [394]:
for x in data:
    x = x.split(" ")
    country = " ".join(x[1:]).replace("*", "")
    cases = int(x[0].replace(",",""))
    case[country] = cases

In [395]:
case = collections.OrderedDict(sorted(case.items()))

In [376]:
df = pd.DataFrame.from_dict(case, orient="index", columns=[':'.join(TIME.split(":")[:-1])]).reset_index()

In [377]:
df = df.rename(columns={"index":"Country"})

In [379]:
df.to_csv("Confirmed_case.csv", index=False)